# Sentiment Analysis of Twitter (now, X) Posts
**By**: Stephen Borja, Justin Ching, Erin Chua, and Zhean Ganituen.

**Dataset**: Hussein, S. (2021). Twitter Sentiments Dataset [Dataset]. Mendeley. https://doi.org/10.17632/Z9ZW7NT5H2.1

**Motivation**: Every minute, social media users generate a large influx of textual data on live events. Performing sentiment analysis on this data provides a real-time view of public perception, enabling quick insights into the general population’s opinions and reactions.

**Goal**: By the end of the project, our goal is to create and compare supervised learning algorithms for sentiment analysis. <br>

## 1 project set up
Set up here the imports for the projects (ensure these are installed via uv and is part of the environment). Furthermore, load the dataset here. Also load the raw dataset here.

In [ ]:
import torch

# Load raw data file
RAW_DATASET = ()

Finished running setup!


## 2 data cleaning
This section discusses the methodology for data cleaning.

---
**References**
1. ()

In [ ]:
pass